# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120084e+02     1.390961e+00
 * time: 0.41573190689086914
     1     1.186827e+01     7.419784e-01
 * time: 1.084571123123169
     2    -1.135208e+01     8.409282e-01
 * time: 1.147965908050537
     3    -3.391223e+01     6.333214e-01
 * time: 1.2347540855407715
     4    -4.739502e+01     4.618503e-01
 * time: 1.3541390895843506
     5    -5.668285e+01     1.868615e-01
 * time: 1.4258639812469482
     6    -5.960154e+01     1.468754e-01
 * time: 1.478977918624878
     7    -6.080969e+01     6.471577e-02
 * time: 1.5321569442749023
     8    -6.132535e+01     4.875280e-02
 * time: 1.5860040187835693
     9    -6.162144e+01     5.965625e-02
 * time: 1.6478080749511719
    10    -6.185182e+01     3.044946e-02
 * time: 1.724614143371582
    11    -6.201144e+01     1.965854e-02
 * time: 1.7782630920410156
    12    -6.208668e+01     1.624904e-02
 * time: 1.8315250873565674
    13    -6.213048e+01     1.381503e-02
 * time: 1.8846769332885

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671056
    AtomicLocal         -18.8557626
    AtomicNonlocal      14.8522618
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485352 
    Xc                  -19.3336812

    total               -62.261666458177